In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [68]:
result = [] # 결과를 저장하기 위한 리스트

for page in range(1, 21): # 1페이지부터 10페이지 까지 정보
    url = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page={}'.format(page) # NAVER Finance
    data = requests.get(url) # url로 부터 정보 받아옴
    pageString = data.text
    bsObj = BeautifulSoup(pageString, "html.parser")
    box_type_l = bsObj.find("div", {"class":"box_type_l"}) # div 영역 추출
    type_2 = box_type_l.find("table", {"class":"type_2"}) # table 영역 추출
    tbody = type_2.find("tbody") # tbody 영역 추출
    trs = tbody.findAll("tr") # tr 영역 추출

    for i in range(0, len(trs)):  # 길이가 27이 아닌 데이터들을 무시             
        if len(trs[i])!=27: 
            continue
        else: 
            tr = trs[i]
            tds = tr.findAll("td")
            number = tds[0].text # 회사 순위 수집
            company = tds[1].text # 회사명 수집
            company_number = tds[1].find("a")["href"][-6:] # 회사 code 수집
            per = tds[10].text # PER 정보 수집
            roe = tds[11].text # ROE 정보 수집
            siga_total = tds[6].text.replace(',','') # 시가 총액 정보 수집
            line = [number, company, company_number, siga_total, per, roe] # 수집한 정보 리스트화
        result.append(line) # 결과 저장하기 위한 리스트에 추가

stock = pd.DataFrame(result)
stock.columns = ['rank', 'company', 'code', 'siga_total', 'PER', 'ROE']

my_data = []
for c in range(0, stock.shape[0]):  
    url = 'https://finance.naver.com/item/main.naver?code={}'.format(stock['code'][c]) # code를 이용해 주식 세부창 주소 획득
    data = requests.get(url) # 사이트 주소 가져오기
    pageString = data.text
    bsObj = BeautifulSoup(pageString, "html.parser")
    section = bsObj.find("div", {"class":"section cop_analysis"}) # div
    if (section == None): 
        pass
    else:
        table = section.find("table", {"class":"tb_type1 tb_num tb_type1_ifrs"})
        tbody = table.find("tbody")
        trs = tbody.findAll("tr")

#    tr = trs[0]
#    tds = tr.findAll("td")
#    a1 = tds[1].text.strip() # 2019 매출액
#    b1 = tds[2].text.strip() # 2020 매출액
#    c1 = tds[3].text.strip() # 2021 매출액
#    d1 = tds[7].text.strip() # 2021년 1분기 매출액 
#    e1 = tds[8].text.strip() # 2021년 2분기 매출액
#    f1 = tds[9].text.strip() # 2021년 3분기 매출액 

    tr2 = trs[1]
    tds2 = tr2.findAll("td")
#    aa1 = tds2[1].text.strip() # 2019 영업이익
#    bb1 = tds2[2].text.strip() # 2020 영업이익
#    cc1 = tds2[3].text.strip() # 2021 영업이익
    dd1 = tds2[7].text.strip().replace(',','') # 2021년 1분기 영업이익
    ee1 = tds2[8].text.strip().replace(',','') # 2021년 2분기 영업이익
#    ff1 = tds2[9].text.strip() # 2021년 3분기 영업이익
    if dd1 == '' or ee1 == '' or dd1 == '-' or ee1 == '-':
        pass
    else:
        gg1 = (int(dd1)+int(ee1))*2

#    tr3 = trs[5]
#    tds3 = tr3.findAll("td")
#    x1 = tds3[1].text.strip() # 2019 ROE
#    y1 = tds3[2].text.strip() # 2020 ROE
#    z1 = tds3[3].text.strip() # 2021 ROE

    temp = [stock['company'][c],stock['code'][c], stock['siga_total'][c],gg1]
    my_data.append(temp)

kospi = pd.DataFrame(my_data)
kospi.columns = ['company', 'code', 'siga_total',
                 '2021_sell']

In [69]:
kospi['siga_total'] = kospi['siga_total'].astype(int)
kospi = kospi[kospi['siga_total'] >= 1000]
kospi = kospi[kospi['2021_sell'] > 0]

multiple = 10

kospi['multiple'] = kospi['siga_total'] / (kospi['2021_sell']*multiple)

kospi = kospi.sort_values('multiple')

kospi

In [74]:
result = [] # 결과를 저장하기 위한 리스트

for page in range(1, 21): # 1페이지부터 10페이지 까지 정보
    url = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page={}'.format(page) # NAVER Finance
    data = requests.get(url) # url로 부터 정보 받아옴
    pageString = data.text
    bsObj = BeautifulSoup(pageString, "html.parser")
    box_type_l = bsObj.find("div", {"class":"box_type_l"}) # div 영역 추출
    type_2 = box_type_l.find("table", {"class":"type_2"}) # table 영역 추출
    tbody = type_2.find("tbody") # tbody 영역 추출
    trs = tbody.findAll("tr") # tr 영역 추출

    for i in range(0, len(trs)):  # 길이가 27이 아닌 데이터들을 무시             
        if len(trs[i])!=27: 
            continue
        else: 
            tr = trs[i]
            tds = tr.findAll("td")
            number = tds[0].text # 회사 순위 수집
            company = tds[1].text # 회사명 수집
            company_number = tds[1].find("a")["href"][-6:] # 회사 code 수집
            per = tds[10].text # PER 정보 수집
            roe = tds[11].text # ROE 정보 수집
            siga_total = tds[6].text.replace(',','') # 시가 총액 정보 수집
            line = [number, company, company_number, siga_total, per, roe] # 수집한 정보 리스트화
        result.append(line) # 결과 저장하기 위한 리스트에 추가

stock = pd.DataFrame(result)
stock.columns = ['rank', 'company', 'code', 'siga_total', 'PER', 'ROE']

my_data = []
for c in range(0, stock.shape[0]):  
    url = 'https://finance.naver.com/item/main.naver?code={}'.format(stock['code'][c]) # code를 이용해 주식 세부창 주소 획득
    data = requests.get(url) # 사이트 주소 가져오기
    pageString = data.text
    bsObj = BeautifulSoup(pageString, "html.parser")
    section = bsObj.find("div", {"class":"section cop_analysis"}) # div
    if (section == None): 
        pass
    else:
        table = section.find("table", {"class":"tb_type1 tb_num tb_type1_ifrs"})
        tbody = table.find("tbody")
        trs = tbody.findAll("tr")

#    tr = trs[0]
#    tds = tr.findAll("td")
#    a1 = tds[1].text.strip() # 2019 매출액
#    b1 = tds[2].text.strip() # 2020 매출액
#    c1 = tds[3].text.strip() # 2021 매출액
#    d1 = tds[7].text.strip() # 2021년 1분기 매출액 
#    e1 = tds[8].text.strip() # 2021년 2분기 매출액
#    f1 = tds[9].text.strip() # 2021년 3분기 매출액 

    tr2 = trs[1]
    tds2 = tr2.findAll("td")
#    aa1 = tds2[1].text.strip() # 2019 영업이익
#    bb1 = tds2[2].text.strip() # 2020 영업이익
#    cc1 = tds2[3].text.strip() # 2021 영업이익
    dd1 = tds2[7].text.strip().replace(',','') # 2021년 1분기 영업이익
    ee1 = tds2[8].text.strip().replace(',','') # 2021년 2분기 영업이익
#    ff1 = tds2[9].text.strip() # 2021년 3분기 영업이익
    if dd1 == '' or ee1 == '' or dd1 == '-' or ee1 == '-':
        pass
    else:
        gg1 = (int(dd1)+int(ee1))*2

#    tr3 = trs[5]
#    tds3 = tr3.findAll("td")
#    x1 = tds3[1].text.strip() # 2019 ROE
#    y1 = tds3[2].text.strip() # 2020 ROE
#    z1 = tds3[3].text.strip() # 2021 ROE

    temp = [stock['company'][c],stock['code'][c], stock['siga_total'][c],gg1]
    my_data.append(temp)

kosdaq = pd.DataFrame(my_data)
kosdaq.columns = ['company', 'code', 'siga_total',
                 '2021_sell']

In [75]:
kosdaq['siga_total'] = kosdaq['siga_total'].astype(int)
kosdaq = kosdaq[kosdaq['siga_total'] >= 1000]
kosdaq = kosdaq[kosdaq['2021_sell'] > 0]

multiple = 10

kosdaq['multiple'] = kosdaq['siga_total'] / (kosdaq['2021_sell']*multiple)

kosdaq = kosdaq.sort_values('multiple')

In [76]:
kosdaq

,company,code,siga_total,2021_sell,multiple
167,다우데이타,032190,5170,13966,0.037018
787,CNH,023460,1254,1060,0.118302
809,심텍홀딩스,036710,1216,932,0.130472
722,매일홀딩스,005990,1413,946,0.149366
82,하림지주,003380,8726,5548,0.157282
...,...,...,...,...,...
769,넥스트BT,065170,1284,2,64.200000
749,한국테크놀로지,053590,1339,2,66.950000
96,파크시스템스,140860,7390,8,92.375000
410,내츄럴엔도텍,168330,2467,2,123.350000
